# Capstone Project Model Solution
In this notebook, we provide a model solution for the capstone project. You can refer to this if you get stuck at any step while building your solution. The model solution guides you to answer the problem statement posed in the capstone project.

The notebook is structured as follows:
1. [Read the Data](#read)
2. [Variable Initialisation](#initialise)
3. [Data Preprocessing](#process)
4. [Fetching and Merging OHLC Data](#fetch_merge)
5. [Generate Sentiment Score](#sentiment)
6. [Backtesting](#backtesting)
7. [Conclusion](#conclusion)

## Import Libraries

In [ ]:
# Import necessary libraries
import os
import re
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
import whisper
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame

# Import utility functions
import sys
sys.path.append('..')
from data_modules import finbert_sa

<a id='read'></a>
## Read the Data

In [ ]:
# Load the CSV file containing timestamps
timestamp_df = pd.read_csv('../data_modules/AMZN_EC_timestamps.csv')
timestamp_df['Meeting Date'] = pd.to_datetime(timestamp_df['EC Date'], format='%m-%d-%Y')

<a id='initialise'></a>
## Variable Initialisation

In [ ]:
# Initialize an empty list to store results
results = []

# Folder containing audio files
audio_folder = '../data_modules'

# API Keys
stock_client = StockHistoricalDataClient("YOUR API KEY",  "YOUR_API_SECRET_KEY")

# Initialize an empty list to store OHLC data
ohlc_data = []

# Create a dataframe 'trade_sheet' to store the trades
trade_sheet = pd.DataFrame()

# Initialise the current_position as '0'
current_position = 0

# Define a variable to store the trade's entry date and time
entry_datetime = ''

# Define a variable to store the long entry price
entry_price = ''

# Define a variable to store the trade's exit date and time
exit_datetime = ''

# Define a variable to store the long exit price
exit_price = ''

# Use 0.1 as a threshold
sentiment_score_threshold = 0.01

<a id='process'></a>
## Data Preprocessing

In [ ]:
# Loop through each row in the CSV
for index, row in timestamp_df.iterrows():
    try:
        # Generate the audio filename
        date_str = row['EC Date']
        audio_filename = f'earnings_call_{date_str}.wav'  # Adjust extension based on your audio file format
        audio_path = os.path.join(audio_folder, audio_filename)

        # Check if the audio file exists
        if not os.path.exists(audio_path):
            print(f"Audio file not found: {audio_filename}")
            continue

        # Transcribe audio using Whisper API
        model = whisper.load_model("base")
        transcript = model.transcribe(audio_path)
        speech_text = transcript['text']  # Extract the transcribed text

        # Clean the text: Fix broken words and remove extra spaces
        speech_text = re.sub(r'\n(?!\s*[A-Z])', '', speech_text)
        speech_text = re.sub(r'\s+', ' ', speech_text)
        speech_text = re.sub(r'\s+([.,!?])', r'\1', speech_text)

        # Split the speech into words
        words = speech_text.split()

        # Calculate the total time for the speech
        start_time = datetime.strptime(row['Start Time'], '%H:%M')
        end_time = datetime.strptime(row['End Time'], '%H:%M:%S')
        total_minutes = int((end_time - start_time).total_seconds() / 60)

        # Calculate words per minute
        words_per_minute = len(words) // total_minutes

        # Generate the rows with 1-minute intervals
        current_time = start_time
        for i in range(total_minutes):
            minute_words = words[i * words_per_minute: (i + 1) * words_per_minute]
            timestamp = datetime.combine(row['Meeting Date'], current_time.time())
            timestamp_utc = pd.to_datetime(timestamp).tz_localize('America/Los_Angeles').tz_convert('UTC')

            results.append({
                'timestamp': timestamp_utc,
                'text': ' '.join(minute_words),
                'video_id': index + 1  # Assuming index serves as unique video/audio ID
            })
            current_time += timedelta(minutes=1)

    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Convert results to a DataFrame
result_df = pd.DataFrame(results)

# Display the dataframe
result_df.head()

,timestamp,text,video_id
0,2024-08-01 21:30:00+00:00,"Thank you for standing by. Good day, everyone,...",1
1,2024-08-01 21:31:00+00:00,"1, 2024 only, and will include forward-looking...",1
2,2024-08-01 21:32:00+00:00,"growth of the internet, online commerce, cloud...",1
3,2024-08-01 21:33:00+00:00,"growth. First, companies have completed the si...",1
4,2024-08-01 21:34:00+00:00,that our unique approach and offerings are res...,1


<a id='fetch_merge'></a>
## Fetching and Merging OHLC Data

In [ ]:
def fetch_ohlc_for_timestamp(timestamp, symbol):
    # Create a StockBarsRequest object to specify the request parameters for fetching OHLC data
    request_params = StockBarsRequest(
        symbol_or_symbols=symbol,               # Specify the symbol for which data is being fetched (in this case, SPY)
        timeframe=TimeFrame.Minute,            # Set the timeframe to 1 minute to get minute-level OHLC data
        start=timestamp,                       # Define the start time for the data request
        end=timestamp + pd.Timedelta(minutes=1)  # Define the end time, 1 minute after the start time
    )

    # Send the request to the stock client and get the OHLC data
    bars = stock_client.get_stock_bars(request_params)

    # Check if the returned DataFrame is empty, indicating no data for the requested timestamp
    if bars.df.empty:
        return None  # Return None if no data is found
    else:
        return bars.df.iloc[0]  # Return the first row of the fetched data if available

# Iterate over each row in the result_df DataFrame
for index, row in result_df.iterrows():
    timestamp = row['timestamp']  # Extract the timestamp from the current row
    symbol = "AMZN"

    # Fetch the OHLC data for the extracted timestamp
    ohlc = fetch_ohlc_for_timestamp(timestamp, symbol)

    # Check if OHLC data was successfully retrieved
    if ohlc is not None:
        # If data is available, append it to the ohlc_data list with relevant OHLC values
        ohlc_data.append({
            'timestamp': timestamp,
            'open': ohlc['open'],
            'high': ohlc['high'],
            'low': ohlc['low'],
            'close': ohlc['close'],
        })
    else:
        # If data is not available, append None for OHLC values
        ohlc_data.append({
            'timestamp': timestamp,
            'open': None,
            'high': None,
            'low': None,
            'close': None,
        })

# Convert the list of OHLC data to a DataFrame
ohlc_df = pd.DataFrame(ohlc_data)

# Merge the OHLC data with the result_df on the Timestamp
merged_df = pd.merge(result_df, ohlc_df, on='timestamp', how='left')

# Print the merged DataFrame
merged_df.head()

,timestamp,text,video_id,open,high,low,close
0,2024-08-01 21:30:00+00:00,"Thank you for standing by. Good day, everyone,...",1,174.8000,174.82,174.5500,174.67
1,2024-08-01 21:31:00+00:00,"1, 2024 only, and will include forward-looking...",1,174.5500,174.80,174.5001,174.70
2,2024-08-01 21:32:00+00:00,"growth of the internet, online commerce, cloud...",1,174.7584,174.90,174.6000,174.70
3,2024-08-01 21:33:00+00:00,"growth. First, companies have completed the si...",1,174.5500,174.75,174.3500,174.66
4,2024-08-01 21:34:00+00:00,that our unique approach and offerings are res...,1,174.5007,174.88,174.5000,174.75


<a id='sentiment'></a>
## Generate Sentiment Score

In [ ]:
# Load the FinBERT sentiment analysis model using the `load_model()` function
finbert_model = finbert_sa.load_model()

scores = finbert_sa.process_sentences(finbert_model, merged_df['text'])

# Add the computed sentiment scores to the DataFrame as a new column 'sentiment_score'
merged_df['sentiment_score'] = scores
merged_df.head()

  0%|          | 0/48 [00:00<?, ?it/s]

,timestamp,text,video_id,open,high,low,close,sentiment_score
0,2024-08-01 21:30:00+00:00,"Thank you for standing by. Good day, everyone,...",1,174.8000,174.82,174.5500,174.67,0.098115
1,2024-08-01 21:31:00+00:00,"1, 2024 only, and will include forward-looking...",1,174.5500,174.80,174.5001,174.70,-0.089332
2,2024-08-01 21:32:00+00:00,"growth of the internet, online commerce, cloud...",1,174.7584,174.90,174.6000,174.70,0.255520
3,2024-08-01 21:33:00+00:00,"growth. First, companies have completed the si...",1,174.5500,174.75,174.3500,174.66,0.483434
4,2024-08-01 21:34:00+00:00,that our unique approach and offerings are res...,1,174.5007,174.88,174.5000,174.75,0.107383


<a id='backtesting'></a>
## Backtesting

In [ ]:
# Iterate over the dates in the dataframe 'data'
for i in merged_df.video_id.unique():
    video = merged_df[merged_df['video_id'] == i]
    video['rolling_sentiment_score'] = video['sentiment_score'].expanding(
    ).mean()

    for timestamp in video.index:

        # We will enter the long position if we are not holding any position and the entry condition is met
        if current_position == 0 and timestamp != video.index[-1]:
            if (video.loc[timestamp]['rolling_sentiment_score']
                    > sentiment_score_threshold):
                print('________________')
                print('opening long position at ', timestamp)

                # Define the variable 'entry_date'
                entry_datetime = timestamp

                # Extract the 'Close price' of the current_date and store it the variable 'entry price'
                entry_price = video.loc[entry_datetime, 'close']

                # Long position is opened so update the current_position to '1'
                current_position = 1

            elif (video.loc[timestamp]['rolling_sentiment_score']
                  < -sentiment_score_threshold):
                print('opening short position at ', timestamp)

                # Define the variable 'entry_date'
                entry_datetime = timestamp

                # Extract the 'Close price' of the current_date and store it the variable 'entry price'
                entry_price = video.loc[entry_datetime, 'close']

                # Long position is opened so update the current_position to '1'
                current_position = -1

        # We will exit the long position if we are holding a long position and the exit condition is met
        elif current_position == 1 and (
            (video.loc[timestamp]['rolling_sentiment_score']
             < -sentiment_score_threshold) or (timestamp == video.index[-1])):

            print('closing long position at ', timestamp)

            # Define the variable 'exit_date'
            exit_datetime = timestamp

            # Extract the 'Close price' on the current_date and store in the variable 'exit price'
            exit_price = video.loc[exit_datetime, 'close']

            # Append the details of this trade to the 'trade_sheet' dataframe
            trade_sheet = trade_sheet.append(
                [(current_position, entry_datetime, entry_price, exit_datetime,
                  exit_price)],
                ignore_index=True)

            # Long position is closed so update the current_position to '0'
            current_position = 0

        # We will exit the long position if we are holding a long position and the exit condition is met
        elif current_position == -1 and (
            (video.loc[timestamp]['rolling_sentiment_score']
             > sentiment_score_threshold) or (timestamp == video.index[-1])):

            print('closing short position at ', timestamp)

            # Define the variable 'exit_date'
            exit_datetime = timestamp

            # Extract the 'Close price' on the current_date and store in the variable 'exit price'
            exit_price = video.loc[exit_datetime, 'close']

            # Append the details of this trade to the 'trade_sheet' dataframe
            trade_sheet = trade_sheet.append(
                [(current_position, entry_datetime, entry_price, exit_datetime,
                  exit_price)],
                ignore_index=True)

            # Long position is closed so update the current_position to '0'
            current_position = 0

# Define the names of columns in 'trade_sheet' dataframe
trade_sheet.columns = [
    'Position', 'Entry Datetime', 'Entry Price', 'Exit Datetime', 'Exit Price'
]
trade_sheet['pnl'] = trade_sheet['Position'] * (trade_sheet['Exit Price'] -
                                                trade_sheet['Entry Price'])

trade_sheet.tail()

________________
opening long position at  0
closing long position at  47


,Position,Entry Datetime,Entry Price,Exit Datetime,Exit Price,pnl
0,1,0,174.67,47,172.462,-2.208


<a id='conclusion'></a>
## Conclusion

In this notebook, we generated the sentiment scores and took trade based on these scores for one earnings call meeting. Similarly, you can do this for multiple meetings and further analyse the performance of the strategy.